In [193]:
import pandas as pd
import numpy as np
import math

In [194]:
data = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Result_Datos\result_meses\Enero.csv",
                   sep=';', encoding='latin-1')

#data = pd.read_csv(r"C:\Users\andre\OneDrive\Documentos\Maestría\Proyecto de Grado\Result_Datos\Datos_Ene_Nov_2023.csv",
#                   sep=';', encoding='latin-1')

df = data[(data['EDAD'].isin(range(18,29))) & (data['ACTIVIDAD_OCUPADA_ULTIMA_SEMANA'].isin([1,2]))].reset_index(drop=True)

In [195]:
df.shape

(6902, 70)

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6902 entries, 0 to 6901
Data columns (total 70 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   MES                                   6902 non-null   int64  
 1   DIRECTORIO                            6902 non-null   int64  
 2   SECUENCIA_P                           6902 non-null   int64  
 3   ORDEN                                 6902 non-null   int64  
 4   AREA_METROPOLITANA                    5203 non-null   float64
 5   ZONA_TERRITORIAL                      6902 non-null   int64  
 6   FACTOR_EXPANSION                      6902 non-null   float64
 7   DPTO                                  6902 non-null   int64  
 8   SEXO                                  6902 non-null   int64  
 9   EDAD                                  6902 non-null   int64  
 10  PARENTESCO_JEFE_DE_HOGAR              6902 non-null   int64  
 11  MADRE_RESIDE_HOGA

### Columnas a descartar

In [197]:
df = df.drop(columns=['AREA_METROPOLITANA',
                       'ALGUNA_VEZ_CAMPESINO',
                       'CONYUGE_VIVE_HOGAR',
                       'TIPO_REGIMEN_SALUD',
                       'QUIEN_PAGA_SALUD',
                       'VALOR_SALUD',
                       'INSTITUCION_PUBLICA_PRIVADA',
                       'USO_COMPARTIDO_SANITARIO',
                       'TIPO_COMBUSTIBLE_COCINAR'])

### Descarte columnas de identificación y peso de muestreo

In [198]:
df = df.drop(columns=['MES','SECUENCIA_P','ORDEN'])

In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6902 entries, 0 to 6901
Data columns (total 58 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   DIRECTORIO                            6902 non-null   int64  
 1   ZONA_TERRITORIAL                      6902 non-null   int64  
 2   FACTOR_EXPANSION                      6902 non-null   float64
 3   DPTO                                  6902 non-null   int64  
 4   SEXO                                  6902 non-null   int64  
 5   EDAD                                  6902 non-null   int64  
 6   PARENTESCO_JEFE_DE_HOGAR              6902 non-null   int64  
 7   MADRE_RESIDE_HOGAR                    6902 non-null   int64  
 8   PADRE_RESIDE_HOGAR                    6902 non-null   int64  
 9   SE_CONSIDERA_CAMPESINO                6902 non-null   int64  
 10  COMUNIDAD_ES_CAMPESINA                6902 non-null   int64  
 11  ETNIA            

### Revisión cuentas financieras

In [200]:
financieros = ['HOGAR_TIENE_CUENTA_CORRIENTE',
'HOGAR_TIENE_CUENTA_AHORROS',
'HOGAR_TIENE_CDT',
'HOGAR_TIENE_PRESTAMO_COMPRA_VIVIENDA',
'HOGAR_TIENE_PRESTAMO_COMPRA_VEHICULO',
'HOGAR_TIENE_PRESTAMO_LIBRE_INVERSION',
'HOGAR_TIENE_TARJETA_CREDITO',
'HOGAR_TIENE_NINGUNO_DE_ESTOS',
'HOGAR_NO_SABE_FINANCIERO',
'HOGAR_TIENE_OTRO_FINANCIERO']

In [201]:
df[financieros] = df[financieros].fillna(0)

In [202]:
df[financieros]

,HOGAR_TIENE_CUENTA_CORRIENTE,HOGAR_TIENE_CUENTA_AHORROS,HOGAR_TIENE_CDT,HOGAR_TIENE_PRESTAMO_COMPRA_VIVIENDA,HOGAR_TIENE_PRESTAMO_COMPRA_VEHICULO,HOGAR_TIENE_PRESTAMO_LIBRE_INVERSION,HOGAR_TIENE_TARJETA_CREDITO,HOGAR_TIENE_NINGUNO_DE_ESTOS,HOGAR_NO_SABE_FINANCIERO,HOGAR_TIENE_OTRO_FINANCIERO
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
6897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6898,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6900,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [203]:
df[financieros].sum(axis=1).value_counts()

1.0    6015
2.0     675
3.0     182
4.0      24
5.0       5
7.0       1
Name: count, dtype: int64

### Revisión de las variables de educación

In [204]:
data['ACTUALMENTE_ESTUDIA'].value_counts()

ACTUALMENTE_ESTUDIA
2.0    52292
1.0    18482
Name: count, dtype: int64

In [205]:
data['MAYOR_NIVEL_EDUCATIVO'].value_counts()

MAYOR_NIVEL_EDUCATIVO
5.0     17544
3.0     17321
4.0     11670
10.0     8720
8.0      4475
1.0      4022
9.0      1826
2.0      1756
6.0      1359
11.0     1337
12.0      568
7.0       106
13.0       69
99.0        1
Name: count, dtype: int64

In [206]:
df = df.drop(columns=['MAYOR_GRADO_ALCANZADO'])

In [207]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6902 entries, 0 to 6901
Data columns (total 57 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   DIRECTORIO                            6902 non-null   int64  
 1   ZONA_TERRITORIAL                      6902 non-null   int64  
 2   FACTOR_EXPANSION                      6902 non-null   float64
 3   DPTO                                  6902 non-null   int64  
 4   SEXO                                  6902 non-null   int64  
 5   EDAD                                  6902 non-null   int64  
 6   PARENTESCO_JEFE_DE_HOGAR              6902 non-null   int64  
 7   MADRE_RESIDE_HOGAR                    6902 non-null   int64  
 8   PADRE_RESIDE_HOGAR                    6902 non-null   int64  
 9   SE_CONSIDERA_CAMPESINO                6902 non-null   int64  
 10  COMUNIDAD_ES_CAMPESINA                6902 non-null   int64  
 11  ETNIA            

### Tratamiento de las variables condicionadas

In [208]:
df['ESTRATO_ENERGIA_ELECTRICA'].value_counts()

ESTRATO_ENERGIA_ELECTRICA
1.0    2808
2.0    2333
3.0    1103
4.0     300
0.0     150
5.0      96
9.0      63
6.0      23
Name: count, dtype: int64

In [209]:
df['ESTRATO_ENERGIA_ELECTRICA'] = df['ESTRATO_ENERGIA_ELECTRICA'].fillna(10)

In [210]:
df['ESTRATO_ENERGIA_ELECTRICA'].value_counts()

ESTRATO_ENERGIA_ELECTRICA
1.0     2808
2.0     2333
3.0     1103
4.0      300
0.0      150
5.0       96
9.0       63
10.0      26
6.0       23
Name: count, dtype: int64

In [211]:
df['TITULO_OBTENIDO'].value_counts()

TITULO_OBTENIDO
2.0     3097
7.0      859
5.0      769
3.0      363
6.0      317
1.0      167
8.0       65
4.0       14
9.0       13
10.0       1
Name: count, dtype: int64

In [212]:
df['TITULO_OBTENIDO'] = df['TITULO_OBTENIDO'].fillna(0)

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6902 entries, 0 to 6901
Data columns (total 57 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   DIRECTORIO                            6902 non-null   int64  
 1   ZONA_TERRITORIAL                      6902 non-null   int64  
 2   FACTOR_EXPANSION                      6902 non-null   float64
 3   DPTO                                  6902 non-null   int64  
 4   SEXO                                  6902 non-null   int64  
 5   EDAD                                  6902 non-null   int64  
 6   PARENTESCO_JEFE_DE_HOGAR              6902 non-null   int64  
 7   MADRE_RESIDE_HOGAR                    6902 non-null   int64  
 8   PADRE_RESIDE_HOGAR                    6902 non-null   int64  
 9   SE_CONSIDERA_CAMPESINO                6902 non-null   int64  
 10  COMUNIDAD_ES_CAMPESINA                6902 non-null   int64  
 11  ETNIA            

## Renombramiento de Palabras

In [214]:
def replace_values(df, column, mapping):
    df[column] = df[column].replace(mapping)
    return df

In [215]:
df.columns

Index(['DIRECTORIO', 'ZONA_TERRITORIAL', 'FACTOR_EXPANSION', 'DPTO', 'SEXO',
       'EDAD', 'PARENTESCO_JEFE_DE_HOGAR', 'MADRE_RESIDE_HOGAR',
       'PADRE_RESIDE_HOGAR', 'SE_CONSIDERA_CAMPESINO',
       'COMUNIDAD_ES_CAMPESINA', 'ETNIA', 'ESTADO_CIVIL', 'AFILIADO_SALUD',
       'DIFICULTAD_OIR', 'DIFICULTAD_HABLAR', 'DIFICULTAD_VER',
       'DIFICULTAD_MOVER_SUBIR_BAJAR', 'DIFICULTAD_AGARRAR',
       'DIFICULTAD_ENTENDER', 'DIFICULTAD_BANARSE_VESTIRSE',
       'DIFICULTAD_RELACIONARSE', 'LEER_ESCRIBIR', 'ACTUALMENTE_ESTUDIA',
       'MAYOR_NIVEL_EDUCATIVO', 'TITULO_OBTENIDO', 'ORIENTACION_SEXUAL',
       'GENERO', 'ACTIVIDAD_OCUPADA_ULTIMA_SEMANA', 'TIPO_VIVIENDA',
       'MATERIAL_PAREDES', 'MATERIAL_PISOS', 'SERVICIOS_ENERGIA_ELECTRICA',
       'ESTRATO_ENERGIA_ELECTRICA', 'SERVICIOS_GAS_NATURAL',
       'SERVICIOS_ALCANTARILLADO', 'SERVICIOS_RECOLECCION_BASURAS',
       'SERVICIOS_ACUEDUCTO', 'NUMERO_HOGARES_VIVIENDA',
       'NUMERO_CUARTOS_VIVIENDA', 'NUMERO_CUARTOS_DORMIR', 'TIP

In [216]:
## Cambios de valor codificado Original
cambios ={
'ZONA_TERRITORIAL':{
    1:'Cabecera',
    2:'Resto'
    },
'DPTO':{
    5: 'ANTIOQUIA',
    8: 'ATLANTICO',
         11: 'BOGOTA',
         13: 'BOLIVAR',
         15: 'BOYACA',
         17: 'CALDAS',
         18: 'CAQUETA',
         19: 'CAUCA',
         20: 'CESAR',
         23: 'CORDOBA',
         25: 'CUNDINAMARCA',
         27: 'CHOCO',
         41: 'HUILA',
         44: 'LA GUAJIRA',
         47: 'MAGDALENA',
         50: 'META',
         52: 'NARIÑO',
         54: 'NORTE DE SANTANDER',
         63: 'QUINDIO',
         66: 'RISALRALDA',
         68: 'SANTANDER',
         70: 'SUCRE',
         73: 'TOLIMA',
         76: 'VALLE',
         81: 'ARAUCA',
         85: 'CASANARE',
         86: 'PUTUMAYO',
         88: 'SAN ANDRES',
         91: 'AMAZONAS',
         94: 'GUAINIA',
         95: 'GUAVIARE',
         97: 'VAUPES',
         99: 'VICHADA'},
'SEXO':{
    1:'Masculino',
    2:'Femenino'
        },
'PARENTESCO_JEFE_DE_HOGAR':{
    1:'Jefe (a) del hogar',
    2:'Pareja, esposo(a), cónyuge, compañero(a)',
    3:'Hijo(a), hijastro(a)',
    4:'Padre o madre',
    5:'Suegro(a)',
    6:'Hermano(a) o hermanastro(a)',
    7:'Yerno o nuera',
    8:'Nieto(a)',
    9:'Otro  pariente',
    10:'Empleado(a) del servicio doméstico y sus parientes',
    11:'Pensionista',
    12:'Trabajador',
    13:'Otro no pariente'
    },
'MADRE_RESIDE_HOGAR':{
    1:'Si',
    2:'No',
    3:'Fallecida'
    },
'PADRE_RESIDE_HOGAR':{
    1:'Si',
    2:'No',
    3:'Fallecido'
    },
'SE_CONSIDERA_CAMPESINO':{
    1:'Si',
    2:'No',
    9:'No informa'
    },
'COMUNIDAD_ES_CAMPESINA':{
    1:'Si',
    2:'No',
    9:'No informa'
    },
'ETNIA':{
    1:'Indígena',
    2:'Gitano (Rom)',
    3:'Raizal del archipiélago',
    4:'Palenquero',
    5:'Negro, mulato (afrodescendiente)',
    6:'Ninguno de los anteriores'
    },
'ESTADO_CIVIL':{
    1:'No esta casado(a) y vive en pareja  hace  menos de dos años',
    2:'No esta  casado (a)  y vive en pareja   hace dos años  o más',
    3:'Esta casado (a)',
    4:'Esta separado (a) o divorciado (a)',
    5:'Esta viudo (a)',
    6:'Esta soltero (a)'
    },
'AFILIADO_SALUD':{
    1:'Si',
    2:'No',
    9:'No sabe, no informa'
    },
'DIFICULTAD_OIR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_HABLAR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_VER':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_MOVER_SUBIR_BAJAR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_AGARRAR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_ENTENDER':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_BANARSE_VESTIRSE':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_RELACIONARSE':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'LEER_ESCRIBIR':{
    1:'Si',
    2:'No'
    },
'ACTUALMENTE_ESTUDIA':{
    1:'Si',
    2:'No'
    },
'MAYOR_NIVEL_EDUCATIVO':{
    1:'Ninguno',
    2:'Preescolar ',
    3:'Básica primaria (1o - 5o)',
    4:'Básica secundaria (6o - 9o)',
    5:'Media académica (Bachillerato clásico)',
    6:'Media técnica (Bachillerato técnico)',
    7:'Normalista',
    8:'Técnica profesional',
    9:'Tecnológica ',
    10:'Universitaria',
    11:'Especialización ',
    12:'Maestría ',
    13:'Doctorado ',
    99:'No sabe, no informa'
},
'TITULO_OBTENIDO':{
    0:'No Aplica',
    1:'Ninguno',
    2:'Media académica (Bachillerato clásico)',
    3:'Media técnica (Bachillerato técnico)',
    4:'Normalista',
    5:'Técnica profesional',
    6:'Tecnológica ',
    7:'Universitaria',
    8:'Especialización ',
    9:'Maestría ',
    10:'Doctorado ',
    99:'No sabe, no informa'
    },
'ORIENTACION_SEXUAL':{
    1:'Hombres',
    2:'Mujeres',
    3:'Ambos sexos',
    4:'Otro'
    },
'GENERO':{
    1:'Hombre',
    2:'Mujer',
    3:'Hombre trans',
    4:'Mujer trans',
    5:'Otro'
    },
'ACTIVIDAD_OCUPADA_ULTIMA_SEMANA':{
    1:1,
    2:0
    },
'TIPO_VIVIENDA':{
    1:'Casa',
    2:'Apartamento',
    3:'Cuarto (s) en inquilinato',
    4:'Cuarto (s) en otro  tipo de estructura',
    5:'Vivienda indígena',
    6:'Otra vivienda (carpa,  vagón, embarcación, cueva, refugio natural, etc.)'
    },
'MATERIAL_PAREDES':{
    1:'Ladrillo, bloque, material prefabricado, piedra',
    2:'Madera pulida',
    3:'Adobe o tapia pisada',
    4:'Bahareque',
    5:'Madera burda, tabla, tablón',
    6:'Guadua',
    7:'Caña, esterilla, otro tipo de material vegetal',
    8:'Zinc, tela, cartón, latas, desechos, plástico',
    9:'Sin paredes'
    },
'MATERIAL_PISOS':{
    1:'Tierra, arena',
    2:'Cemento, gravilla',
    3:'Madera burda, tabla, tablón, otro vegetal',
    4:'Baldosín, ladrillo, vinisol, otros materiales sintéticos',
    5:'Mármol',
    6:'Madera pulida',
    7:'Alfombra o tapete de pared a pared'
    },
'SERVICIOS_ENERGIA_ELECTRICA':{
    1:'Si',
    2:'No'
    },
'ESTRATO_ENERGIA_ELECTRICA':{
    0:'Conexión Pirata',
    1:'Bajo - Bajo',
    2:'Bajo',
    3:'Medio - Bajo',
    4:'Medio',
    5:'Medio - Alto',
    6:'Alto',
    9:'No sabe o cuenta con Planta Eléctrica',
    10:'No tiene servicio eléctrico'
    },
'SERVICIOS_GAS_NATURAL':{
    1:'Si',
    2:'No'
    },
'SERVICIOS_ALCANTARILLADO':{
    1:'Si',
    2:'No'
    },
'SERVICIOS_RECOLECCION_BASURAS':{
    1:'Si',
    2:'No'
    },
'SERVICIOS_ACUEDUCTO':{
    1:'Si',
    2:'No'
    },
'TIPO_SANITARIO':{
    1:'Inodoro conectado a alcantarillado',
    2:'Inodoro conectado a pozo séptico',
    3:'Inodoro sin conexión',
    4:'Letrina',
    5:'Bajamar',
    6:'No tiene servicio sanitario'
    },
'COMO_ELIMINA_BASURA':{
    1:'Por recolección pública o privada',
    2:'La tiran a un río, quebrada, caño o laguna',
    3:'La tiran a un patio, lote, zanja o baldío',
    4:'La queman o entierran',
    5:'La eliminan de otra forma'
    },
'DONDE_OBTIENE_AGUA':{
    1:'De acueducto por tubería',
    2:'De otra fuente por tubería',
    3:'De pozo con bomba',
    4:'De pozo sin bomba, aljibe, jagüey o barreno',
    5:'Aguas lluvias',
    6:'Rio, quebrada, nacimiento o manantial ',
    7:'De pila pública',
    8:'Carrotanque',
    9:'Aguatero',
    10:'Agua embotellada o en bolsa'
    },
'DONDE_PREPARA_ALIMENTOS':{
    1:'En un cuarto usado sólo para cocinar?',
    2:'En un cuarto usado también para dormir?',
    3:'En una sala comedor con lavaplatos?',
    4:'En una sala comedor sin lavaplatos?',
    5:'En un patio, corredor, enramada, al aire libre?',
    6:'En ninguna parte, no preparan alimentos?'
    },
'TIPO_OCUPACION_VIVIENDA':{
    1:'Propia, totalmente pagada',
    2:'Propia, la están pagando',
    3:'En arriendo o subarriendo',
    4:'En usufructo',
    5:'Posesión sin título',
    6:'Propiedad colectiva',
    7:'Otra'
    },
'HOGAR_TIENE_CUENTA_CORRIENTE':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_CUENTA_AHORROS':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_CDT':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_PRESTAMO_COMPRA_VIVIENDA':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_PRESTAMO_COMPRA_VEHICULO':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_PRESTAMO_LIBRE_INVERSION':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_TARJETA_CREDITO':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_OTRO_FINANCIERO':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_NINGUNO_DE_ESTOS':{
    1:'Si',
    0:'No'
    },
'HOGAR_NO_SABE_FINANCIERO':{
    1:'Si',
    0:'No'
    },
}

In [ ]:
## Cambios de valor codificado con la agrupación de niveles
cambios_agrupado ={
'ZONA_TERRITORIAL':{
    1:'Cabecera',
    2:'Resto'
    },
'DPTO':{
    5: 'ANTIOQUIA',
    8: 'ATLANTICO',
         11: 'BOGOTA',
         13: 'BOLIVAR',
         15: 'BOYACA',
         17: 'CALDAS',
         18: 'CAQUETA',
         19: 'CAUCA',
         20: 'CESAR',
         23: 'CORDOBA',
         25: 'CUNDINAMARCA',
         27: 'CHOCO',
         41: 'HUILA',
         44: 'LA GUAJIRA',
         47: 'MAGDALENA',
         50: 'META',
         52: 'NARIÑO',
         54: 'NORTE DE SANTANDER',
         63: 'QUINDIO',
         66: 'RISALRALDA',
         68: 'SANTANDER',
         70: 'SUCRE',
         73: 'TOLIMA',
         76: 'VALLE',
         81: 'ARAUCA',
         85: 'CASANARE',
         86: 'PUTUMAYO',
         88: 'SAN ANDRES',
         91: 'AMAZONAS',
         94: 'GUAINIA',
         95: 'GUAVIARE',
         97: 'VAUPES',
         99: 'VICHADA'},
'SEXO':{
    1:'Masculino',
    2:'Femenino'
        },
'PARENTESCO_JEFE_DE_HOGAR':{
    1:'Jefe (a) del hogar',
    2:'Pareja, esposo(a), cónyuge, compañero(a)',
    3:'Hijo(a), hijastro(a)',
    4:'Padre o madre',
    5:'Suegro(a)',
    6:'Hermano(a) o hermanastro(a)',
    7:'Yerno o nuera',
    8:'Nieto(a)',
    9:'Otro  pariente',
    10:'Empleado(a) del servicio doméstico y sus parientes',
    11:'Pensionista',
    12:'Trabajador',
    13:'Otro no pariente'
    },
'MADRE_RESIDE_HOGAR':{
    1:'Si',
    2:'No',
    3:'Fallecida'
    },
'PADRE_RESIDE_HOGAR':{
    1:'Si',
    2:'No',
    3:'Fallecido'
    },
'SE_CONSIDERA_CAMPESINO':{
    1:'Si',
    2:'No',
    9:'No informa'
    },
'COMUNIDAD_ES_CAMPESINA':{
    1:'Si',
    2:'No',
    9:'No informa'
    },
'ETNIA':{
    1:'Indígena',
    2:'Gitano (Rom)',
    3:'Raizal del archipiélago',
    4:'Palenquero',
    5:'Negro, mulato (afrodescendiente)',
    6:'Ninguno de los anteriores'
    },
'ESTADO_CIVIL':{
    1:'No esta casado(a) y vive en pareja  hace  menos de dos años',
    2:'No esta  casado (a)  y vive en pareja   hace dos años  o más',
    3:'Esta casado (a)',
    4:'Esta separado (a) o divorciado (a)',
    5:'Esta viudo (a)',
    6:'Esta soltero (a)'
    },
'AFILIADO_SALUD':{
    1:'Si',
    2:'No',
    9:'No sabe, no informa'
    },
'DIFICULTAD_OIR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_HABLAR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_VER':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_MOVER_SUBIR_BAJAR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_AGARRAR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_ENTENDER':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_BANARSE_VESTIRSE':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_RELACIONARSE':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'LEER_ESCRIBIR':{
    1:'Si',
    2:'No'
    },
'ACTUALMENTE_ESTUDIA':{
    1:'Si',
    2:'No'
    },
'MAYOR_NIVEL_EDUCATIVO':{
    1:'Ninguno',
    2:'Preescolar ',
    3:'Básica primaria (1o - 5o)',
    4:'Básica secundaria (6o - 9o)',
    5:'Media académica (Bachillerato clásico)',
    6:'Media técnica (Bachillerato técnico)',
    7:'Normalista',
    8:'Técnica profesional',
    9:'Tecnológica ',
    10:'Universitaria',
    11:'Especialización ',
    12:'Maestría ',
    13:'Doctorado ',
    99:'No sabe, no informa'
},
'TITULO_OBTENIDO':{
    0:'No Aplica',
    1:'Ninguno',
    2:'Media académica (Bachillerato clásico)',
    3:'Media técnica (Bachillerato técnico)',
    4:'Normalista',
    5:'Técnica profesional',
    6:'Tecnológica ',
    7:'Universitaria',
    8:'Especialización ',
    9:'Maestría ',
    10:'Doctorado ',
    99:'No sabe, no informa'
    },
'ORIENTACION_SEXUAL':{
    1:'Hombres',
    2:'Mujeres',
    3:'Ambos sexos',
    4:'Otro'
    },
'GENERO':{
    1:'Hombre',
    2:'Mujer',
    3:'Hombre trans',
    4:'Mujer trans',
    5:'Otro'
    },
'ACTIVIDAD_OCUPADA_ULTIMA_SEMANA':{
    1:1,
    2:0
    },
'TIPO_VIVIENDA':{
    1:'Casa',
    2:'Apartamento',
    3:'Cuarto (s) en inquilinato',
    4:'Cuarto (s) en otro  tipo de estructura',
    5:'Vivienda indígena',
    6:'Otra vivienda (carpa,  vagón, embarcación, cueva, refugio natural, etc.)'
    },
'MATERIAL_PAREDES':{
    1:'Ladrillo, bloque, material prefabricado, piedra',
    2:'Madera pulida',
    3:'Adobe o tapia pisada',
    4:'Bahareque',
    5:'Madera burda, tabla, tablón',
    6:'Guadua',
    7:'Caña, esterilla, otro tipo de material vegetal',
    8:'Zinc, tela, cartón, latas, desechos, plástico',
    9:'Sin paredes'
    },
'MATERIAL_PISOS':{
    1:'Tierra, arena',
    2:'Cemento, gravilla',
    3:'Madera burda, tabla, tablón, otro vegetal',
    4:'Baldosín, ladrillo, vinisol, otros materiales sintéticos',
    5:'Mármol',
    6:'Madera pulida',
    7:'Alfombra o tapete de pared a pared'
    },
'SERVICIOS_ENERGIA_ELECTRICA':{
    1:'Si',
    2:'No'
    },
'ESTRATO_ENERGIA_ELECTRICA':{
    0:'Conexión Pirata',
    1:'Bajo - Bajo',
    2:'Bajo',
    3:'Medio - Bajo',
    4:'Medio',
    5:'Medio - Alto',
    6:'Alto',
    9:'No sabe o cuenta con Planta Eléctrica',
    10:'No tiene servicio eléctrico'
    },
'SERVICIOS_GAS_NATURAL':{
    1:'Si',
    2:'No'
    },
'SERVICIOS_ALCANTARILLADO':{
    1:'Si',
    2:'No'
    },
'SERVICIOS_RECOLECCION_BASURAS':{
    1:'Si',
    2:'No'
    },
'SERVICIOS_ACUEDUCTO':{
    1:'Si',
    2:'No'
    },
'TIPO_SANITARIO':{
    1:'Inodoro conectado a alcantarillado',
    2:'Inodoro conectado a pozo séptico',
    3:'Inodoro sin conexión',
    4:'Letrina',
    5:'Bajamar',
    6:'No tiene servicio sanitario'
    },
'COMO_ELIMINA_BASURA':{
    1:'Por recolección pública o privada',
    2:'La tiran a un río, quebrada, caño o laguna',
    3:'La tiran a un patio, lote, zanja o baldío',
    4:'La queman o entierran',
    5:'La eliminan de otra forma'
    },
'DONDE_OBTIENE_AGUA':{
    1:'De acueducto por tubería',
    2:'De otra fuente por tubería',
    3:'De pozo con bomba',
    4:'De pozo sin bomba, aljibe, jagüey o barreno',
    5:'Aguas lluvias',
    6:'Rio, quebrada, nacimiento o manantial ',
    7:'De pila pública',
    8:'Carrotanque',
    9:'Aguatero',
    10:'Agua embotellada o en bolsa'
    },
'DONDE_PREPARA_ALIMENTOS':{
    1:'En un cuarto usado sólo para cocinar?',
    2:'En un cuarto usado también para dormir?',
    3:'En una sala comedor con lavaplatos?',
    4:'En una sala comedor sin lavaplatos?',
    5:'En un patio, corredor, enramada, al aire libre?',
    6:'En ninguna parte, no preparan alimentos?'
    },
'TIPO_OCUPACION_VIVIENDA':{
    1:'Propia, totalmente pagada',
    2:'Propia, la están pagando',
    3:'En arriendo o subarriendo',
    4:'En usufructo',
    5:'Posesión sin título',
    6:'Propiedad colectiva',
    7:'Otra'
    },
'HOGAR_TIENE_CUENTA_CORRIENTE':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_CUENTA_AHORROS':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_CDT':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_PRESTAMO_COMPRA_VIVIENDA':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_PRESTAMO_COMPRA_VEHICULO':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_PRESTAMO_LIBRE_INVERSION':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_TARJETA_CREDITO':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_OTRO_FINANCIERO':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_NINGUNO_DE_ESTOS':{
    1:'Si',
    0:'No'
    },
'HOGAR_NO_SABE_FINANCIERO':{
    1:'Si',
    0:'No'
    },
}

In [217]:
for col in cambios.keys():
    df_pfinal = replace_values(df, col, cambios[col])

In [218]:
from unidecode import unidecode

In [219]:
cat_variables = df_pfinal.drop(columns='ACTIVIDAD_OCUPADA_ULTIMA_SEMANA').select_dtypes(include=['object']).columns.to_list()
num_variables = df_pfinal.drop(columns='ACTIVIDAD_OCUPADA_ULTIMA_SEMANA').select_dtypes(include=['int64','float64']).columns.to_list()

In [220]:
for col in cat_variables:
    df[col] = df[col].apply(lambda x: unidecode(x))

In [221]:
df_pfinal.to_csv('df_transformado_ENE.csv', index=False, sep=';', encoding='latin1')

In [222]:
import sys
sys.path.append('c:\\Users\\andre\\OneDrive\\Documentos\\Maestría\\Proyecto de Grado')

In [223]:
from data_module.pruebas import calculate_woe_iv

In [224]:
iv_df = pd.DataFrame(columns=['IV'])

# Calculate WOE and IV for each variable in the categorical dataframe
for column in cat_variables:
    woe, iv = calculate_woe_iv(df_pfinal, column, 'ACTIVIDAD_OCUPADA_ULTIMA_SEMANA')
    sum_iv = sum([i for i in iv.values()])
    iv_df.loc[column, 'IV'] = sum_iv

# Display the resulting dataframe
print("Resultados del Cálculo de IV")
display(iv_df.sort_values(by='IV', ascending=False))

print("Variables con IV mayor a 0.1, es decir, predictores aceptables")
display(iv_df[iv_df['IV'] > 0.1].sort_values(by='IV', ascending=False))

iv_columns = list(iv_df[iv_df['IV'] > 0.1].sort_values(by='IV', ascending=False).index)

Resultados del Cálculo de IV


,IV
PARENTESCO_JEFE_DE_HOGAR,0.309034
MADRE_RESIDE_HOGAR,0.21639
ESTADO_CIVIL,0.192598
DPTO,0.155438
TITULO_OBTENIDO,0.076905
PADRE_RESIDE_HOGAR,0.072577
TIPO_OCUPACION_VIVIENDA,0.064025
MAYOR_NIVEL_EDUCATIVO,0.05213
TIPO_VIVIENDA,0.036521
DONDE_OBTIENE_AGUA,0.027622


Variables con IV mayor a 0.1, es decir, predictores aceptables


,IV
PARENTESCO_JEFE_DE_HOGAR,0.309034
MADRE_RESIDE_HOGAR,0.21639
ESTADO_CIVIL,0.192598
DPTO,0.155438


In [225]:
iv_columns = iv_columns + ['GENERO','ETNIA','SEXO','ESTRATO_ENERGIA_ELECTRICA']

In [226]:
iv_columns

['PARENTESCO_JEFE_DE_HOGAR',
 'MADRE_RESIDE_HOGAR',
 'ESTADO_CIVIL',
 'DPTO',
 'GENERO',
 'ETNIA',
 'SEXO',
 'ESTRATO_ENERGIA_ELECTRICA']

In [227]:
df_pfinal[num_variables + ['ACTIVIDAD_OCUPADA_ULTIMA_SEMANA']].corr()

,DIRECTORIO,FACTOR_EXPANSION,EDAD,NUMERO_HOGARES_VIVIENDA,NUMERO_CUARTOS_VIVIENDA,NUMERO_CUARTOS_DORMIR,NUMERO_PERSONAS_HOGAR,ACTIVIDAD_OCUPADA_ULTIMA_SEMANA
DIRECTORIO,1.000000,0.075983,-0.005558,-0.018943,-0.007997,-0.019637,0.006070,0.025213
FACTOR_EXPANSION,0.075983,1.000000,0.006668,-0.006845,-0.033627,-0.023252,-0.035783,0.036880
EDAD,-0.005558,0.006668,1.000000,-0.001559,0.002459,-0.061347,-0.091382,0.088229
NUMERO_HOGARES_VIVIENDA,-0.018943,-0.006845,-0.001559,1.000000,-0.078384,-0.083216,-0.032475,0.018170
NUMERO_CUARTOS_VIVIENDA,-0.007997,-0.033627,0.002459,-0.078384,1.000000,0.774530,0.384103,-0.076042
NUMERO_CUARTOS_DORMIR,-0.019637,-0.023252,-0.061347,-0.083216,0.774530,1.000000,0.583132,-0.090014
NUMERO_PERSONAS_HOGAR,0.006070,-0.035783,-0.091382,-0.032475,0.384103,0.583132,1.000000,-0.082967
ACTIVIDAD_OCUPADA_ULTIMA_SEMANA,0.025213,0.036880,0.088229,0.018170,-0.076042,-0.090014,-0.082967,1.000000


In [228]:
num_variables

['DIRECTORIO',
 'FACTOR_EXPANSION',
 'EDAD',
 'NUMERO_HOGARES_VIVIENDA',
 'NUMERO_CUARTOS_VIVIENDA',
 'NUMERO_CUARTOS_DORMIR',
 'NUMERO_PERSONAS_HOGAR']

In [229]:
df_pfinal.head()

,DIRECTORIO,ZONA_TERRITORIAL,FACTOR_EXPANSION,DPTO,SEXO,EDAD,PARENTESCO_JEFE_DE_HOGAR,MADRE_RESIDE_HOGAR,PADRE_RESIDE_HOGAR,SE_CONSIDERA_CAMPESINO,...,HOGAR_TIENE_CUENTA_AHORROS,HOGAR_TIENE_CDT,HOGAR_TIENE_PRESTAMO_COMPRA_VIVIENDA,HOGAR_TIENE_PRESTAMO_COMPRA_VEHICULO,HOGAR_TIENE_PRESTAMO_LIBRE_INVERSION,HOGAR_TIENE_TARJETA_CREDITO,HOGAR_TIENE_OTRO_FINANCIERO,HOGAR_TIENE_NINGUNO_DE_ESTOS,HOGAR_NO_SABE_FINANCIERO,NUMERO_PERSONAS_HOGAR
0,7309060,Cabecera,1580.677273,CUNDINAMARCA,Masculino,27,"Pareja, esposo(a), conyuge, companero(a)",No,No,Si,...,Si,No,No,No,No,No,No,No,No,3
1,7309064,Resto,324.598262,RISALRALDA,Masculino,18,"Hijo(a), hijastro(a)",Si,No,Si,...,Si,No,No,No,No,No,No,No,No,5
2,7309067,Resto,167.457366,RISALRALDA,Femenino,25,"Hijo(a), hijastro(a)",No,Si,Si,...,Si,No,No,No,No,No,No,No,No,2
3,7309073,Resto,202.189536,RISALRALDA,Femenino,27,"Hijo(a), hijastro(a)",Si,No,Si,...,Si,No,No,No,No,No,No,No,No,6
4,7309073,Resto,202.189536,RISALRALDA,Femenino,23,"Hijo(a), hijastro(a)",Si,No,Si,...,Si,No,No,No,No,No,No,No,No,6


In [230]:
iv_columns = iv_columns + num_variables

In [231]:
X = df_pfinal[iv_columns]
y = df_pfinal['ACTIVIDAD_OCUPADA_ULTIMA_SEMANA']

In [232]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.linear_model import LogisticRegression

Existen variables categóricas en donde la variable cuenta con al menos una categoría en donde esta aparece una única vez, por ejemplo, para la variable dificultad_oir, el valor 'No puede hacerlo' aparece una única vez, esto representa un problema para el entrenamiento del conjunto de datos, así, para este caso de ejemplo se van a eliminar aquellos registros que cuentan con un único valor

In [233]:
def remove_unique_records(df, column):
    value_counts = df[column].value_counts()
    unique_values = value_counts[value_counts == 1].index
    df = df[~df[column].isin(unique_values)].reset_index(drop=True)
    return df


In [234]:
df_cut = df_pfinal.copy()

In [235]:
for col in cat_variables:
    df_cut = remove_unique_records(df_cut, col)

In [236]:
X = df_cut[iv_columns]
y = df_cut['ACTIVIDAD_OCUPADA_ULTIMA_SEMANA']

In [237]:
from sklearn.model_selection import train_test_split

In [238]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [239]:
cat_variables = X.select_dtypes(include=['object']).columns.to_list()
num_variables = X.select_dtypes(include=['int64','float64']).columns.to_list()

## Balanceo de la clase minoritaria a través del Metodo SMOTENC

In [240]:
scaler_1 = StandardScaler()
X_train_scaled = scaler_1.fit_transform(X_train[num_variables])

## X_train_scaled to_ Dataframe
X_train_scaled = pd.DataFrame(X_train_scaled, columns=num_variables, index=X_train.index)

X_train_to_smote = pd.concat([X_train_scaled, X_train[cat_variables]], axis=1)

In [241]:
X_train_to_smote.head()

,DIRECTORIO,FACTOR_EXPANSION,EDAD,NUMERO_HOGARES_VIVIENDA,NUMERO_CUARTOS_VIVIENDA,NUMERO_CUARTOS_DORMIR,NUMERO_PERSONAS_HOGAR,PARENTESCO_JEFE_DE_HOGAR,MADRE_RESIDE_HOGAR,ESTADO_CIVIL,DPTO,GENERO,ETNIA,SEXO,ESTRATO_ENERGIA_ELECTRICA
5542,1.020848,-0.392277,0.386081,-0.143432,0.611409,0.913163,0.069208,"Pareja, esposo(a), conyuge, companero(a)",No,No esta casado (a) y vive en pareja hace d...,SANTANDER,Hombre,Ninguno de los anteriores,Masculino,Medio
358,-1.562497,-0.562045,0.386081,-0.143432,0.611409,-0.187101,-0.455174,Otro pariente,No,Esta soltero (a),CAUCA,Hombre,Ninguno de los anteriores,Masculino,Bajo
2119,-0.688582,0.160749,1.060119,-0.143432,1.525870,2.013427,3.739880,Otro no pariente,No,No esta casado (a) y vive en pareja hace d...,ATLANTICO,Hombre,Ninguno de los anteriores,Masculino,Conexion Pirata
5813,1.186453,1.538812,-1.299014,-0.143432,-0.303052,-0.187101,0.069208,"Hijo(a), hijastro(a)",Si,Esta soltero (a),HUILA,Mujer,Ninguno de los anteriores,Femenino,Bajo
1907,-0.792843,-0.643197,-0.287957,-0.143432,-0.303052,-1.287364,-1.503937,Jefe (a) del hogar,No,Esta soltero (a),VAUPES,Hombre,Ninguno de los anteriores,Masculino,Bajo


In [242]:
cat_smote = [True if x in cat_variables else False for x in X_train_to_smote.columns]

In [243]:
balance = SMOTENC(categorical_features=cat_smote, random_state=123)

In [244]:
X_train_balanced, y_train = balance.fit_resample(X_train_to_smote, y_train)

In [245]:
X_train_balanced

,DIRECTORIO,FACTOR_EXPANSION,EDAD,NUMERO_HOGARES_VIVIENDA,NUMERO_CUARTOS_VIVIENDA,NUMERO_CUARTOS_DORMIR,NUMERO_PERSONAS_HOGAR,PARENTESCO_JEFE_DE_HOGAR,MADRE_RESIDE_HOGAR,ESTADO_CIVIL,DPTO,GENERO,ETNIA,SEXO,ESTRATO_ENERGIA_ELECTRICA
0,1.020848,-0.392277,0.386081,-0.143432,0.611409,0.913163,0.069208,"Pareja, esposo(a), conyuge, companero(a)",No,No esta casado (a) y vive en pareja hace d...,SANTANDER,Hombre,Ninguno de los anteriores,Masculino,Medio
1,-1.562497,-0.562045,0.386081,-0.143432,0.611409,-0.187101,-0.455174,Otro pariente,No,Esta soltero (a),CAUCA,Hombre,Ninguno de los anteriores,Masculino,Bajo
2,-0.688582,0.160749,1.060119,-0.143432,1.525870,2.013427,3.739880,Otro no pariente,No,No esta casado (a) y vive en pareja hace d...,ATLANTICO,Hombre,Ninguno de los anteriores,Masculino,Conexion Pirata
3,1.186453,1.538812,-1.299014,-0.143432,-0.303052,-0.187101,0.069208,"Hijo(a), hijastro(a)",Si,Esta soltero (a),HUILA,Mujer,Ninguno de los anteriores,Femenino,Bajo
4,-0.792843,-0.643197,-0.287957,-0.143432,-0.303052,-1.287364,-1.503937,Jefe (a) del hogar,No,Esta soltero (a),VAUPES,Hombre,Ninguno de los anteriores,Masculino,Bajo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9009,-0.942872,-0.365311,-0.961995,-0.143432,-1.217513,-0.863428,1.320018,"Hijo(a), hijastro(a)",Si,Esta soltero (a),CESAR,Hombre,Ninguno de los anteriores,Masculino,Bajo - Bajo
9010,0.803756,1.007224,0.284790,-0.143432,1.571677,-0.187101,0.042941,"Hijo(a), hijastro(a)",Si,Esta soltero (a),NORTE DE SANTANDER,Hombre,Ninguno de los anteriores,Masculino,Bajo
9011,0.557059,-0.492817,-0.287957,-0.143432,-0.303052,-0.187101,-0.494315,Jefe (a) del hogar,No,Esta soltero (a),CORDOBA,Mujer,Ninguno de los anteriores,Femenino,Bajo - Bajo
9012,-1.126534,-0.557125,-0.780587,-0.143432,-1.920858,-1.287364,-1.100617,Jefe (a) del hogar,No,Esta soltero (a),GUAVIARE,Hombre,Ninguno de los anteriores,Masculino,Bajo - Bajo


In [246]:
X_train_inversed = scaler_1.inverse_transform(X_train_balanced[num_variables])

X_train_inversed = pd.DataFrame(X_train_inversed, columns=num_variables, index=X_train_balanced.index)

for col in X_train_inversed:
    X_train_inversed[col] = X_train_inversed[col].astype(int)

X_train_balanced = pd.concat([X_train_inversed, X_train_balanced[cat_variables]], axis=1)

X_train_balanced

,DIRECTORIO,FACTOR_EXPANSION,EDAD,NUMERO_HOGARES_VIVIENDA,NUMERO_CUARTOS_VIVIENDA,NUMERO_CUARTOS_DORMIR,NUMERO_PERSONAS_HOGAR,PARENTESCO_JEFE_DE_HOGAR,MADRE_RESIDE_HOGAR,ESTADO_CIVIL,DPTO,GENERO,ETNIA,SEXO,ESTRATO_ENERGIA_ELECTRICA
0,7332238,310,25,1,4,3,4,"Pareja, esposo(a), conyuge, companero(a)",No,No esta casado (a) y vive en pareja hace d...,SANTANDER,Hombre,Ninguno de los anteriores,Masculino,Medio
1,7310508,117,25,1,4,2,3,Otro pariente,No,Esta soltero (a),CAUCA,Hombre,Ninguno de los anteriores,Masculino,Bajo
2,7317859,939,27,1,5,4,11,Otro no pariente,No,No esta casado (a) y vive en pareja hace d...,ATLANTICO,Hombre,Ninguno de los anteriores,Masculino,Conexion Pirata
3,7333631,2506,20,1,3,2,4,"Hijo(a), hijastro(a)",Si,Esta soltero (a),HUILA,Mujer,Ninguno de los anteriores,Femenino,Bajo
4,7316982,25,23,1,3,1,1,Jefe (a) del hogar,No,Esta soltero (a),VAUPES,Hombre,Ninguno de los anteriores,Masculino,Bajo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9009,7315720,341,21,1,2,1,6,"Hijo(a), hijastro(a)",Si,Esta soltero (a),CESAR,Hombre,Ninguno de los anteriores,Masculino,Bajo - Bajo
9010,7330411,1901,24,1,5,2,3,"Hijo(a), hijastro(a)",Si,Esta soltero (a),NORTE DE SANTANDER,Hombre,Ninguno de los anteriores,Masculino,Bajo
9011,7328336,196,23,1,3,2,2,Jefe (a) del hogar,No,Esta soltero (a),CORDOBA,Mujer,Ninguno de los anteriores,Femenino,Bajo - Bajo
9012,7314175,123,21,1,1,1,1,Jefe (a) del hogar,No,Esta soltero (a),GUAVIARE,Hombre,Ninguno de los anteriores,Masculino,Bajo - Bajo


In [247]:
y_train.value_counts()  

ACTIVIDAD_OCUPADA_ULTIMA_SEMANA
1.0    4507
0.0    4507
Name: count, dtype: int64

In [248]:
cat_variables

['PARENTESCO_JEFE_DE_HOGAR',
 'MADRE_RESIDE_HOGAR',
 'ESTADO_CIVIL',
 'DPTO',
 'GENERO',
 'ETNIA',
 'SEXO',
 'ESTRATO_ENERGIA_ELECTRICA']

In [249]:
preproccessing = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_variables),
        ('cat', OneHotEncoder(), cat_variables)
    ])

In [250]:
processing_pipeline = imbpipeline(steps=[('preproccessing', preproccessing)])

In [251]:
processing_pipeline.fit(X_train_balanced)

Pipeline(steps=[('preproccessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['DIRECTORIO',
                                                   'FACTOR_EXPANSION', 'EDAD',
                                                   'NUMERO_HOGARES_VIVIENDA',
                                                   'NUMERO_CUARTOS_VIVIENDA',
                                                   'NUMERO_CUARTOS_DORMIR',
                                                   'NUMERO_PERSONAS_HOGAR']),
                                                 ('cat', OneHotEncoder(),
                                                  ['PARENTESCO_JEFE_DE_HOGAR',
                                                   'MADRE_RESIDE_HOGAR',
                                                   'ESTADO_CIVIL', 'DPTO',
                                                   'GENERO', 'ETNIA', 'SEXO',
                                                   'ESTRATO_ENERGIA_ELECTRICA'])]))])

In [252]:
X_train_processed = processing_pipeline.transform(X_train_balanced)

In [253]:
X_train_dataframe = pd.DataFrame(X_train_processed.toarray(), columns=processing_pipeline.get_feature_names_out(), index=X_train_balanced.index)
X_train_dataframe

,num__DIRECTORIO,num__FACTOR_EXPANSION,num__EDAD,num__NUMERO_HOGARES_VIVIENDA,num__NUMERO_CUARTOS_VIVIENDA,num__NUMERO_CUARTOS_DORMIR,num__NUMERO_PERSONAS_HOGAR,cat__PARENTESCO_JEFE_DE_HOGAR_Empleado(a) del servicio domestico y sus parientes,cat__PARENTESCO_JEFE_DE_HOGAR_Hermano(a) o hermanastro(a),"cat__PARENTESCO_JEFE_DE_HOGAR_Hijo(a), hijastro(a)",...,cat__SEXO_Masculino,cat__ESTRATO_ENERGIA_ELECTRICA_Alto,cat__ESTRATO_ENERGIA_ELECTRICA_Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_Bajo - Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_Conexion Pirata,cat__ESTRATO_ENERGIA_ELECTRICA_Medio,cat__ESTRATO_ENERGIA_ELECTRICA_Medio - Alto,cat__ESTRATO_ENERGIA_ELECTRICA_Medio - Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_No sabe o cuenta con Planta Electrica,cat__ESTRATO_ENERGIA_ELECTRICA_No tiene servicio electrico
0,1.073682,-0.361912,0.513879,-0.129994,0.625862,0.911727,0.072202,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,-1.553557,-0.537247,0.513879,-0.129994,0.625862,-0.241073,-0.464342,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.664794,0.209519,1.180848,-0.129994,1.575933,2.064527,3.828007,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.242101,1.633100,-1.153545,-0.129994,-0.324209,-0.241073,0.072202,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.770826,-0.620827,-0.153091,-0.129994,-0.324209,-1.393872,-1.537429,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9009,-0.923407,-0.333749,-0.820060,-0.129994,-1.274279,-1.393872,1.145289,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9010,0.852791,1.083472,0.180394,-0.129994,1.575933,-0.241073,-0.464342,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9011,0.601916,-0.465478,-0.153091,-0.129994,-0.324209,-0.241073,-1.000885,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9012,-1.110203,-0.531796,-0.820060,-0.129994,-2.224350,-1.393872,-1.537429,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [254]:
df_train = X_train_dataframe.copy()
df_train['Y'] = y_train
df_train.to_csv('..\\Result_Datos\\Processed_data\\df_train.csv', encoding = 'latin1', sep = ';', index=False)

In [255]:
import statsmodels as sm

In [256]:
X_train_dataframe = sm.tools.add_constant(X_train_dataframe)

In [257]:
X_train_dataframe

,const,num__DIRECTORIO,num__FACTOR_EXPANSION,num__EDAD,num__NUMERO_HOGARES_VIVIENDA,num__NUMERO_CUARTOS_VIVIENDA,num__NUMERO_CUARTOS_DORMIR,num__NUMERO_PERSONAS_HOGAR,cat__PARENTESCO_JEFE_DE_HOGAR_Empleado(a) del servicio domestico y sus parientes,cat__PARENTESCO_JEFE_DE_HOGAR_Hermano(a) o hermanastro(a),...,cat__SEXO_Masculino,cat__ESTRATO_ENERGIA_ELECTRICA_Alto,cat__ESTRATO_ENERGIA_ELECTRICA_Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_Bajo - Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_Conexion Pirata,cat__ESTRATO_ENERGIA_ELECTRICA_Medio,cat__ESTRATO_ENERGIA_ELECTRICA_Medio - Alto,cat__ESTRATO_ENERGIA_ELECTRICA_Medio - Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_No sabe o cuenta con Planta Electrica,cat__ESTRATO_ENERGIA_ELECTRICA_No tiene servicio electrico
0,1.0,1.073682,-0.361912,0.513879,-0.129994,0.625862,0.911727,0.072202,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,-1.553557,-0.537247,0.513879,-0.129994,0.625862,-0.241073,-0.464342,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,-0.664794,0.209519,1.180848,-0.129994,1.575933,2.064527,3.828007,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.242101,1.633100,-1.153545,-0.129994,-0.324209,-0.241073,0.072202,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,-0.770826,-0.620827,-0.153091,-0.129994,-0.324209,-1.393872,-1.537429,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9009,1.0,-0.923407,-0.333749,-0.820060,-0.129994,-1.274279,-1.393872,1.145289,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9010,1.0,0.852791,1.083472,0.180394,-0.129994,1.575933,-0.241073,-0.464342,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9011,1.0,0.601916,-0.465478,-0.153091,-0.129994,-0.324209,-0.241073,-1.000885,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9012,1.0,-1.110203,-0.531796,-0.820060,-0.129994,-2.224350,-1.393872,-1.537429,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [258]:
from statsmodels.discrete.discrete_model import Logit

In [259]:
logit_mod = Logit(y_train, X_train_dataframe)

In [260]:
X_train_dataframe

,const,num__DIRECTORIO,num__FACTOR_EXPANSION,num__EDAD,num__NUMERO_HOGARES_VIVIENDA,num__NUMERO_CUARTOS_VIVIENDA,num__NUMERO_CUARTOS_DORMIR,num__NUMERO_PERSONAS_HOGAR,cat__PARENTESCO_JEFE_DE_HOGAR_Empleado(a) del servicio domestico y sus parientes,cat__PARENTESCO_JEFE_DE_HOGAR_Hermano(a) o hermanastro(a),...,cat__SEXO_Masculino,cat__ESTRATO_ENERGIA_ELECTRICA_Alto,cat__ESTRATO_ENERGIA_ELECTRICA_Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_Bajo - Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_Conexion Pirata,cat__ESTRATO_ENERGIA_ELECTRICA_Medio,cat__ESTRATO_ENERGIA_ELECTRICA_Medio - Alto,cat__ESTRATO_ENERGIA_ELECTRICA_Medio - Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_No sabe o cuenta con Planta Electrica,cat__ESTRATO_ENERGIA_ELECTRICA_No tiene servicio electrico
0,1.0,1.073682,-0.361912,0.513879,-0.129994,0.625862,0.911727,0.072202,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,-1.553557,-0.537247,0.513879,-0.129994,0.625862,-0.241073,-0.464342,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,-0.664794,0.209519,1.180848,-0.129994,1.575933,2.064527,3.828007,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.242101,1.633100,-1.153545,-0.129994,-0.324209,-0.241073,0.072202,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,-0.770826,-0.620827,-0.153091,-0.129994,-0.324209,-1.393872,-1.537429,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9009,1.0,-0.923407,-0.333749,-0.820060,-0.129994,-1.274279,-1.393872,1.145289,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9010,1.0,0.852791,1.083472,0.180394,-0.129994,1.575933,-0.241073,-0.464342,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9011,1.0,0.601916,-0.465478,-0.153091,-0.129994,-0.324209,-0.241073,-1.000885,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9012,1.0,-1.110203,-0.531796,-0.820060,-0.129994,-2.224350,-1.393872,-1.537429,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [261]:
logit_res = logit_mod.fit()

         Current function value: 0.564913
         Iterations: 35


LinAlgError: Singular matrix

In [ ]:
logit_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  Logit Regression Results                                 
===========================================================================================
Dep. Variable:     ACTIVIDAD_OCUPADA_ULTIMA_SEMANA   No. Observations:                 9014
Model:                                       Logit   Df Residuals:                     8900
Method:                                        MLE   Df Model:                          113
Date:                             Mon, 19 Feb 2024   Pseudo R-squ.:                  0.2668
Time:                                     15:17:09   Log-Likelihood:                -4580.8
converged:                                   False   LL-Null:                       -6248.0
Covariance Type:                         nonrobust   LLR p-value:                     0.000
===============================================================================================================================================================
                                                                                                  coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                           5.4447        nan        nan        nan         nan         nan
num__EDAD                                                                                       0.3038      0.031      9.839      0.000       0.243       0.364
num__NUMERO_HOGARES_VIVIENDA                                                                    0.0823      0.029      2.805      0.005       0.025       0.140
num__NUMERO_CUARTOS_VIVIENDA                                                                    0.1204      0.045      2.662      0.008       0.032       0.209
num__NUMERO_CUARTOS_DORMIR                                                                      0.0223      0.050      0.448      0.654      -0.075       0.120
num__NUMERO_PERSONAS_HOGAR                                                                      0.0143      0.036      0.397      0.691      -0.056       0.085
cat__PARENTESCO_JEFE_DE_HOGAR_Empleado(a) del servicio domestico y sus parientes               11.7223   1.79e+06   6.54e-06      1.000   -3.51e+06    3.51e+06
cat__PARENTESCO_JEFE_DE_HOGAR_Hermano(a) o hermanastro(a)                                      -4.5417   2.72e+06  -1.67e-06      1.000   -5.33e+06    5.33e+06
cat__PARENTESCO_JEFE_DE_HOGAR_Hijo(a), hijastro(a)                                             -6.0606   2.41e+06  -2.52e-06      1.000   -4.72e+06    4.72e+06
cat__PARENTESCO_JEFE_DE_HOGAR_Jefe (a) del hogar                                               -5.2895   2.15e+06  -2.47e-06      1.000    -4.2e+06     4.2e+06
cat__PARENTESCO_JEFE_DE_HOGAR_Nieto(a)                                                         -5.4748   1.63e+06  -3.36e-06      1.000   -3.19e+06    3.19e+06
cat__PARENTESCO_JEFE_DE_HOGAR_Otro  pariente                                                   -5.1836   1.68e+06  -3.09e-06      1.000   -3.29e+06    3.29e+06
cat__PARENTESCO_JEFE_DE_HOGAR_Otro no pariente                                                 -4.6259   3.96e+05  -1.17e-05      1.000   -7.77e+05    7.77e+05
cat__PARENTESCO_JEFE_DE_HOGAR_Pareja, esposo(a), conyuge, companero(a)                         -4.9621   1.37e+06  -3.63e-06      1.000   -2.68e+06    2.68e+06
cat__PARENTESCO_JEFE_DE_HOGAR_Pensionista                                                       7.4138   1.45e+06   5.13e-06      1.000   -2.83e+06    2.83e+06
cat__PARENTESCO_JEFE_DE_HOGAR_Suegro(a)                                                        12.0155   2.05e+06   5.86e-06      1.000   -4.02e+06    4.02e+06
cat__PARENTESCO_JEFE_DE_HOGAR_Trabajador                                            

In [ ]:
X_test_tr = preproccessing.transform(X_test).toarray()
X_test_df = pd.DataFrame(X_test_tr, columns=preproccessing.get_feature_names_out())
#X_test_df
X_test_df['Y'] = y_test.reset_index(drop=True)
X_test_cv = sm.tools.add_constant(X_test_tr)

X_test_df.to_csv('..\\Result_Datos\\Processed_data\\df_test.csv', encoding = 'latin1', sep = ';', index=False)

In [ ]:
X_test_df

,num__EDAD,num__NUMERO_HOGARES_VIVIENDA,num__NUMERO_CUARTOS_VIVIENDA,num__NUMERO_CUARTOS_DORMIR,num__NUMERO_PERSONAS_HOGAR,cat__PARENTESCO_JEFE_DE_HOGAR_Empleado(a) del servicio domestico y sus parientes,cat__PARENTESCO_JEFE_DE_HOGAR_Hermano(a) o hermanastro(a),"cat__PARENTESCO_JEFE_DE_HOGAR_Hijo(a), hijastro(a)",cat__PARENTESCO_JEFE_DE_HOGAR_Jefe (a) del hogar,cat__PARENTESCO_JEFE_DE_HOGAR_Nieto(a),...,cat__ESTRATO_ENERGIA_ELECTRICA_Alto,cat__ESTRATO_ENERGIA_ELECTRICA_Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_Bajo - Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_Conexion Pirata,cat__ESTRATO_ENERGIA_ELECTRICA_Medio,cat__ESTRATO_ENERGIA_ELECTRICA_Medio - Alto,cat__ESTRATO_ENERGIA_ELECTRICA_Medio - Bajo,cat__ESTRATO_ENERGIA_ELECTRICA_No sabe o cuenta con Planta Electrica,cat__ESTRATO_ENERGIA_ELECTRICA_No tiene servicio electrico,Y
0,-1.485925,-0.129994,-0.323171,-0.239238,-0.463827,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.486537,-0.129994,-0.323171,-0.239238,0.073707,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.512850,-0.129994,3.492240,3.225939,1.148774,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.845980,4.633294,-2.230877,-1.394298,-1.538894,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,-0.153408,-0.129994,-0.323171,-0.239238,-1.001361,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,-0.153408,-0.129994,0.630682,0.915821,0.073707,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1376,0.512850,-0.129994,-0.323171,-0.239238,0.073707,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1377,0.179721,-0.129994,-0.323171,-0.239238,-1.001361,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1378,1.179109,-0.129994,-0.323171,-0.239238,1.686308,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
y_pred = logit_res.predict(X_test_cv)
y_pred_cl = np.round(y_pred, 0)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, recall_score, precision_score

In [ ]:
confusion_matrix(y_test, y_pred_cl)

array([[107, 137],
       [328, 808]], dtype=int64)

In [ ]:
accuracy_score(y_test, y_pred_cl)

0.6630434782608695

In [ ]:
precision_score(y_test, y_pred_cl)

0.855026455026455

In [ ]:
recall_score(y_test, y_pred_cl)

0.7112676056338029

In [ ]:
f1_score(y_test, y_pred_cl)

0.7765497357039884